## Create documents

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/data.csv')

In [3]:
documents = df.to_dict(orient='records')

In [4]:
documents

[{'id': 1,
  'attraction': 'Geirangerfjord',
  'activity_type': 'Sightseeing / Nature',
  'county': 'Møre og Romsdal',
  'time_to_visit': 'May, June, July, August, September',
  'description': 'One of Norway’s most iconic fjords, surrounded by steep cliffs, waterfalls, and small farms. It is a UNESCO World Heritage Site and offers boat tours, kayaking, and breathtaking viewpoints such as Dalsnibba.'},
 {'id': 2,
  'attraction': 'Preikestolen (Pulpit Rock)',
  'activity_type': 'Hiking',
  'county': 'Rogaland',
  'time_to_visit': 'May, June, July, August, September, October',
  'description': 'A flat plateau on a steep cliff 604 meters above Lysefjord. The hike takes 2–4 hours each way and rewards visitors with one of Norway’s most photographed panoramic views.'},
 {'id': 3,
  'attraction': 'Trolltunga',
  'activity_type': 'Hiking',
  'county': 'Vestland',
  'time_to_visit': 'June, July, August, September',
  'description': 'A famous rock ledge jutting horizontally out from the mountain.

In [5]:
df.columns

Index(['id', 'attraction', 'activity_type', 'county', 'time_to_visit',
       'description'],
      dtype='object')

## Implement minsearch

In [6]:
import minsearch

In [7]:
index = minsearch.Index(
    text_fields = ['attraction', 'activity_type', 'county', 'time_to_visit',
       'description'],
    keyword_fields=['id']
)

In [8]:
index.fit(documents)

In [9]:
query = 'give me one hiking tour in Rogaland'

In [10]:
index.search(query, num_results=5)

[{'id': 2,
  'attraction': 'Preikestolen (Pulpit Rock)',
  'activity_type': 'Hiking',
  'county': 'Rogaland',
  'time_to_visit': 'May, June, July, August, September, October',
  'description': 'A flat plateau on a steep cliff 604 meters above Lysefjord. The hike takes 2–4 hours each way and rewards visitors with one of Norway’s most photographed panoramic views.'},
 {'id': 11,
  'attraction': 'Kjeragbolten',
  'activity_type': 'Hiking',
  'county': 'Rogaland',
  'time_to_visit': 'June, July, August, September',
  'description': 'A massive boulder wedged between cliffs 984 meters above Lysefjord. Adventurous hikers can step onto the rock for dramatic photos, while the hike also offers views of waterfalls and mountains.'},
 {'id': 31,
  'attraction': 'Kjerag Waterfalls',
  'activity_type': 'Nature / Hiking',
  'county': 'Rogaland',
  'time_to_visit': 'June, July, August, September',
  'description': 'Massive waterfalls cascading into Lysefjord. Often combined with the Kjeragbolten hike, 

In [11]:
import openai
from openai import OpenAI

In [12]:
import os
from dotenv import load_dotenv
load_dotenv
api_key = os.getenv("OPENAI_API_KEY")

In [13]:
client = OpenAI()

response = client.chat.completions.create(
    model='gpt-5-mini',
    messages=[{"role": "user", "content": query}]
)

response.choices[0].message.content

'Preikestolen (Pulpit Rock) — classic day hike in Rogaland\n\nQuick facts\n- Location: Lysefjorden area, near Tau / Jørpeland (Strand municipality)\n- Distance: ~7.6 km round trip (≈3.8 km each way)\n- Elevation gain: ~350 m (plateau sits ~604 m above fjord)\n- Time: 3–5 hours round trip (depends on fitness and stops)\n- Difficulty: Moderate (well-marked trail, some steep/rocky sections)\n- Fee/permits: No permit. Parking and ferries cost money.\n\nRoute overview\nStart at the Preikestolen parking / Preikestolen Fjellstue trailhead. The well-maintained trail goes over a mix of roots, rocks and wooden stairs, with a steady climb through pine/heath and open rock slabs near the top. The final approach leads onto the broad cliff plateau with stunning views over Lysefjorden.\n\nHow to get there\n- From Stavanger: take the ferry to Tau, then bus or taxi to the Preikestolen parking/fjellstue. Or drive to the parking area (paid parking).\n- Follow signs for “Preikestolen”/“Preikestolen Fjellst

## RAG flow

In [14]:
query = 'give me one hiking in Rogaland'

In [15]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=5
    )

    return results

In [16]:
documents[0]

{'id': 1,
 'attraction': 'Geirangerfjord',
 'activity_type': 'Sightseeing / Nature',
 'county': 'Møre og Romsdal',
 'time_to_visit': 'May, June, July, August, September',
 'description': 'One of Norway’s most iconic fjords, surrounded by steep cliffs, waterfalls, and small farms. It is a UNESCO World Heritage Site and offers boat tours, kayaking, and breathtaking viewpoints such as Dalsnibba.'}

In [17]:
prompt_template = """
You're a tourist guide in Norway. Answer the QUESTION based on the CONTEXT from our attractions database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()


entry_template = """
attraction: {attraction}
activity_type: {activity_type}
county: {county}
time_to_visit: {time_to_visit}
description: {description}
""".strip()

def build_prompt(query, search_results):
    context = ""

    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [18]:
search_results=search(query)

In [19]:
prompt = build_prompt(query, search_results)
print(prompt)

You're a tourist guide in Norway. Answer the QUESTION based on the CONTEXT from our attractions database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: give me one hiking in Rogaland

CONTEXT:
attraction: Preikestolen (Pulpit Rock)
activity_type: Hiking
county: Rogaland
time_to_visit: May, June, July, August, September, October
description: A flat plateau on a steep cliff 604 meters above Lysefjord. The hike takes 2–4 hours each way and rewards visitors with one of Norway’s most photographed panoramic views.

attraction: Kjeragbolten
activity_type: Hiking
county: Rogaland
time_to_visit: June, July, August, September
description: A massive boulder wedged between cliffs 984 meters above Lysefjord. Adventurous hikers can step onto the rock for dramatic photos, while the hike also offers views of waterfalls and mountains.

attraction: Kjerag Waterfalls
activity_type: Nature / Hiking
county: Rogaland
time_to_visit: June, July, August, September
description: Mas

In [20]:
import openai
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()

In [21]:
def llm(prompt):
    response = client.chat.completions.create(
        model = 'gpt-5-mini',
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [22]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [23]:
answer = rag('I want to visit a beautifull fjord in Norway in December')
print(answer)

Great choice — for December I recommend the Hurtigruten Coastal Voyage. It runs year‑round (including December), sails between Bergen and Kirkenes, and offers breathtaking fjords plus the chance to see the northern lights in winter. 

Note: Balestrand is a picturesque village on the Sognefjord known for its artists’ villas and fjord hotels, but its listed best time to visit is May–September.
